## 要件
- 一行目にソートボタン
- 二行目にフィルターブロック
  - 選択式のドロップダウン
  - 値範囲指定の入力ボックス
- Pythonからデータを埋め込み

<pre>
project/  
├─ data_to_html.py      ← Pythonスクリプト（Jinja2でHTML生成）  
├─ templates/  
│   ├─ base_251020.html ← Jinja2テンプレート
│   └─ table.html
├─ static/  
│   ├─ style/  
│   │   ├─ table.css  
│   │   ├─ filter.css  
│   │   └─ sort.css  
│   └─ js/  
│       └─ table.js  
└─ output/
     └─ table.html          ← Pythonで生成されるブラウザ用HTML
</pre>

In [5]:
from jinja2 import Environment, FileSystemLoader

# --- データ定義 ---
data = [
    {"name": "Alice", "age": 25, "city": "Tokyo"},
    {"name": "Bob", "age": 30, "city": "Osaka"},
    {"name": "Charlie", "age": 22, "city": "Nagoya"},
    {"name": "David", "age": 28, "city": "Tokyo"},
]

# --- 列情報: filterタイプ、候補値、型 ---
columns = [
    {"key": "name", "label": "名前", "filter": "select",
     "options": sorted(set(r["name"] for r in data)), "type": "string"},
    {"key": "age", "label": "年齢", "filter": "range", "type": "number"},
    {"key": "city", "label": "都市", "filter": "select",
     "options": sorted(set(r["city"] for r in data)), "type": "string"},
]

# --- Jinja2テンプレート環境の準備 ---
env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("table.html")

# --- レンダリングして output.html に書き出す ---
html_output = template.render(data=data, columns=columns, title='Sort & Filter')
with open("output/table.html", "w", encoding="utf-8") as f:
    f.write(html_output)

print("table.html を生成しました。ブラウザで開けます。")

table.html を生成しました。ブラウザで開けます。
